In [1]:
import os
import io
import pyarrow as pa
import pyarrow.dataset as ds
import duckdb
import polars as pl
from memory_profiler import profile 
from datetime import datetime
%load_ext memory_profiler
%load_ext line_profiler

In [2]:
# data ingest and sort
data = ds.dataset("~/data/trade_taq2019.arrow", format = "arrow")

# construct scanner to scanner
data = ds.Scanner.from_dataset(data)

# connect duckdb to database
con = duckdb.connect()

In [3]:
def window(data,con):
    
    # window query
    query = "SELECT * FROM data WHERE Symbol = 'AAPL' AND Sale_Condition LIKE '%O%'"
    
    result = con.query(query).df()

    return result

%memit m=window(data,con)
m

peak memory: 346.15 MiB, increment: 169.52 MiB


,Time,Exchange,Symbol,Sale_Condition,Trade_Volume,Trade_Price,Trade_Stop_Stock_Indicator,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Source_of_Trade,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator
0,2019-10-07 09:30:00.562307,Q,AAPL,@O X,223839,226.26,NaN,0,10250,356,N,,2019-10-07 09:30:00.562284,NaN,1


In [4]:
def agg(data,con):
     
    # aggregated timebucket query 
    query = "SELECT ANY_VALUE(Time),SUM(Trade_Volume),Symbol FROM data GROUP BY round(extract('epoch' from Time) / 60), Symbol ORDER BY round(extract('epoch' from Time) / 60)"
    
    result = con.query(query).df()
    
    return result

%memit m=agg(data,con)
m

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

peak memory: 10568.36 MiB, increment: 10376.40 MiB


,"any_value(""Time"")","sum(""Trade_Volume"")",Symbol
0,2019-10-07 04:00:00.029165,187.0,SCHW
1,2019-10-07 04:00:00.396934,92.0,BEN
2,2019-10-07 04:00:00.401074,1.0,RJF
3,2019-10-07 04:00:00.015525,100.0,VXX
4,2019-10-07 04:00:00.024208,650.0,BTI
...,...,...,...
1596718,2019-10-07 20:00:00.007501,100.0,INCO
1596719,2019-10-07 20:00:00.005393,100.0,WBIL
1596720,2019-10-07 20:00:00.007860,100.0,NBO
1596721,2019-10-07 20:00:00.003468,2887.0,IEUR


In [5]:
def scanner(data,con):

    #aggregating query
    query = "SELECT Symbol,Exchange,AVG(Trade_Price),AVG(Trade_Volume) FROM data GROUP BY Symbol,Exchange ORDER BY AVG(Trade_Price) DESC"
    
    result = con.query(query).df()
    
    return result

%memit m=scanner(data,con)
m

peak memory: 9988.54 MiB, increment: 180.02 MiB


,Symbol,Exchange,"avg(""Trade_Price"")","avg(""Trade_Volume"")"
0,BRK A,K,310979.198571,1.000000
1,BRK A,N,310927.540000,1.114754
2,BRK A,P,310875.000000,1.000000
3,BRK A,D,310857.104533,1.444444
4,BRK A,Z,310752.624000,1.000000
...,...,...,...,...
82965,SNOA W,Q,0.004360,520.500000
82966,APDN W,Q,0.004224,513.793103
82967,SNOA W,P,0.004200,100.000000
82968,APDN W,P,0.002500,7000.000000
